<a href="https://colab.research.google.com/github/ryderwishart/nlp-model-experiments/blob/main/Greek_BERT_one_word_ln.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [ ]:
# uncomment if you want to quickly delete unwanted files from previous model training
import shutil
#shutil.rmtree('/content/MyDrive/MyDrive/output')

Follow the general approach in this POS-tagging notebook: https://github.com/Kyubyong/nlp_made_easy/blob/master/Pos-tagging%20with%20Bert%20Fine-tuning.ipynb

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import unicodedata
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
from collections import Counter
import json
import os
import sys

In [3]:
if 'macula-greek.tsv' not in [path for path in os.listdir()]:
    !wget -q 'https://raw.githubusercontent.com/Clear-Bible/macula-greek/main/Nestle1904/TSV/macula-greek.tsv'
if 'ln_int_dict.txt' not in [path for path in os.listdir()]:
    !wget -q 'https://raw.githubusercontent.com/ryderwishart/nlp-model-experiments/main/data/ln_int_dict.txt'
if 'lemma_ln_dict.txt' not in [path for path in os.listdir()]:
    !wget -q 'https://raw.githubusercontent.com/ryderwishart/nlp-model-experiments/main/data/lemma_ln_dict.txt'
if 'ln_pos_dict.txt' not in [path for path in os.listdir()]:
    !wget -q 'https://raw.githubusercontent.com/ryderwishart/nlp-model-experiments/main/data/ln_pos_dict.txt'

#Load in data and dictionaries

In [28]:
data ='macula-greek.tsv'
ln_int_dict = 'ln_int_dict.txt'
lemma_ln_dict = 'lemma_ln_dict.txt'
ln_pos_dict = 'ln_pos_dict.txt'

In [6]:
with open(ln_int_dict) as f:
    ln_int_dict = json.load(f)

with open(lemma_ln_dict, encoding="utf8") as f:
    lemma_ln_dict = json.load(f)

with open(ln_pos_dict) as f:
    ln_pos_dict = json.load(f)

In [20]:
# previous input factors: text, lemma
input = 'text'
# previous output factors: ln, type, role, class(pos)
output = 'type'

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [29]:
data = pd.read_csv(data, sep='\t', keep_default_na=False, encoding='utf-8')
data = data[[input,output]]
data.head()

,text,type
0,Βίβλος,common
1,γενέσεως,common
2,Ἰησοῦ,proper
3,Χριστοῦ,proper
4,υἱοῦ,common


In [30]:
if output == 'ln':
  lns = data[output].tolist()

  for i in range(len(lns)):
    lns[i] = ln_int_dict[lns[i]]

  data = data.drop(output, axis=1)
  data[output] = lns


if output == 'type':
  type_int = {}

  label = 0
  types = data[output].tolist()
  for i in range(len(types)):
    if types[i] not in type_int:
      type_int[types[i]] = label
      types[i] = type_int[types[i]]
      label += 1
    else:
      types[i] = type_int[types[i]]

  data = data.drop(output, axis=1)
  data[output] = types

if output == 'class':
  pos_int = {}

  label = 0
  pos = data[output].tolist()
  for i in range(len(pos)):
    if pos[i] not in pos_int:
      pos_int[pos[i]] = label
      pos[i] = pos_int[pos[i]]
      label += 1
    else:
      pos[i] = pos_int[pos[i]]

  data = data.drop(output, axis=1)
  data[output] = pos

data.head()

,text,type
0,Βίβλος,0
1,γενέσεως,0
2,Ἰησοῦ,1
3,Χριστοῦ,1
4,υἱοῦ,0


In [23]:
if output == 'ln':
  sem_doms = data['ln'].nunique()

# BERT MODEL

In [24]:
''' 
other models tested include: 
nlpaueb/bert-base-greek-uncased-v1, 
pranaydeeps/Ancient-Greek-BERT, 
bert-base-multilingual-cased, 
bert-base-multilingual-uncased
'''
pretrained = "pranaydeeps/Ancient-Greek-BERT"

In [25]:
tokenizer = BertTokenizer.from_pretrained(pretrained)
if output == 'ln':
  model = BertForSequenceClassification.from_pretrained(pretrained, num_labels = sem_doms)

if output == 'type':
  model = BertForSequenceClassification.from_pretrained(pretrained, num_labels = len(type_int))

if output == 'class':
  model = BertForSequenceClassification.from_pretrained(pretrained, num_labels = len(pos_int))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pranaydeeps/Ancient-Greek-BERT and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
model = model.to('cuda')

In [31]:
X = list(data[input])
y = list(data[output])
# 70/20/10 split for train, val, and test
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2) # 80/20
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.125) # 70/10
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [32]:
print(f'{len(X_train)} items in the training set.')
print(f'{len(X_val)} items in the validation set.')
print(f'{len(X_test)} items in the test set.')

96445 items in the training set.
27556 items in the validation set.
13778 items in the test set.


In [33]:
 # Create dataset
 class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [34]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [35]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='weighted')
    precision = precision_score(y_true=labels, y_pred=pred, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=pred, average='weighted')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [36]:
args = TrainingArguments(
    # Change if you want the checkpoints to go somewhere else
    output_dir="/content/MyDrive/MyDrive/output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=2048,
    per_device_eval_batch_size=2048,
    num_train_epochs=3,
    seed=0,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)


In [37]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


OutOfMemoryError: ignored

#TEST

In [ ]:
X_test[:5]

In [ ]:
# true values
y_test[:5]

In [ ]:
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)
test_dataset = Dataset(X_test_tokenized)

In [ ]:
# CHANGE MODEL PATH TO BEST PERFORMING MODEL
model_path = "/content/output/checkpoint-500"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=sem_doms, ignore_mismatched_sizes=True)

In [ ]:
test_trainer = Trainer(model)

In [ ]:
raw_pred, _, _ = test_trainer.predict(test_dataset)

In [ ]:
y_pred = np.argmax(raw_pred, axis=1)

In [ ]:
predictions = y_pred.tolist()

In [ ]:
predictions[:5]

In [ ]:
int_ln_dict = {v: k for k, v in ln_int_dict.items()}

In [ ]:
true_values = y_test.copy()
predicted_values = predictions.copy()

In [ ]:
if output != 'ln':
  sys.exit()

#Split Up Data into Groups

Analyze performance between content words (common nouns, proper nouns, adjectives, verbs, adverbs) and function words (i.e. conjunctions and prepositions)


In [ ]:
content = []
function = []

if output == 'ln':
  for i in range(len(true_values)):
    a = int_ln_dict[str(y_test[i])]
    b = int_ln_dict[str(predicted_values[i])]
    if ln_pos_dict[a] in ('noun', 'adj', 'verb'):
      content.append(tuple((a, b)))
    else:
      function.append(tuple((a, b)))   
      
if output == 'class':
  for i in range(len(pos_true)):
    pos_true[i] = inv_pos_int[pos_true[i]]
    pos_predictions[i] = inv_pos_int[pos_predictions[i]]
      if pos_true[i] in ('noun', 'adjective', 'verb'):
        content.append(tuple((X_test[i], pos_true[i], pos_predictions[i])))
      else:
        function.append(tuple((X_test[i], pos_true[i], pos_predictions[i])))

Analyze performance between words seen in the training dataset and those only seen in the test dataset

In [ ]:
in_train = []
out_train = []

for i in range(len(true_values)):
  a = int_ln_dict[str(true_values[i])]
  b = int_ln_dict[str(predicted_values[i])]
  if (true_values[i] in y_train) or (true_values[i] in y_val):
    in_train.append(tuple((a, b)))
  else:
    out_train.append(tuple((a, b)))

In [1]:
pos_list = ['noun', 'adj', 'verb', 'adv', 'det', 'conj', 'prep', 'pron', 'ptcl', 
            'num', 'intj']

role_list = ['CL','np','adjp','IO','S','O','VC','O2','vp','V','verb','conj',
             'det','prep','ADV','pp','advp','nump','intj','ptcl','pron','P']

In [ ]:
ambiguous_ln = []
ambiguous_words = []
unambiguous_ln = []

for i in range(len(X_test)):
  a = int_ln_dict[str(true_values[i])]
  b = int_ln_dict[str(predicted_values[i])]
  domains = lemma_ln_dict[X_test[i]]
  if len(domains) > 1:
    ambiguous_ln.append(tuple((a, b)))
    ambiguous_words.append(X_test[i])
  else:
    unambiguous_ln.append(tuple((a, b)))

# Functions

In [ ]:
def generate_incorrect_lists(set_list: list) -> list:
  incorrect_list = []

  for i in set_list:
    if i[0] != i[1]:
      incorrect_list.append(i)

  return incorrect_list

In [ ]:
def generate_incorrect_ambiguity_list(set_list: list, ambiguous_words: list) -> list:
  incorrect_list = []
  corresponding_domains = []

  for i in range(len(set_list)):
    if set_list[i][0] != set_list[i][1]:
      incorrect_list.append(set_list[i])
      corresponding_domains.append(lemma_ln_dict[ambiguous_words[i]])

  return incorrect_list, corresponding_domains

In [ ]:
def count_all_wrong(wrong_list: list, total: list):
  wrong_count = len(wrong_list)
  all_count = len(total)
  accuracy = wrong_count/all_count

  print(f'{wrong_count} domains incorrectly identified out of {all_count}.')
  print(f'{round(accuracy, 4)*100}% of domains incorrectly identified')

In [ ]:
def count_wrong_ln_by_pos(pos_list: list, wrong_list: list) -> dict:
  key_list = pos_list
  wrong_pos_counts = {k:0 for k in key_list}
  
  for i in wrong_list:
    pos = ln_pos_dict[i[0][0]]
    wrong_pos_counts[pos] += i[1]
  
  return wrong_pos_counts

In [ ]:
def count_all_pos(pos_list: list, set_list: list) -> dict:
  key_list = pos_list
  all_pos_counts = {k:0 for k in key_list}

  try:
    for i in set_list:
      all_pos_counts[ln_pos_dict[i[0]]] += 1
  except:
    for i in set_list:
      all_pos_counts[ln_pos_dict[int_ln_dict[str(i)]]] += 1
    
  return all_pos_counts

In [ ]:
def create_pos_stats(wrong_pos_counts: dict, all_pos_counts: dict) -> list:   
  stats = []
  for i in wrong_pos_counts:
    if all_pos_counts[i] == 0:
          stats.append('{} out of {} {}s incorrect (0.0)'
              .format(wrong_pos_counts[i], all_pos_counts[i], i))
    else:
        stats.append('{} out of {} {}s incorrect ({})'
              .format(wrong_pos_counts[i], all_pos_counts[i], i, 
                      round((wrong_pos_counts[i]/all_pos_counts[i]), 4)))

  return stats

In [ ]:
def return_relevant_stats(pos_type: str, stats: list):
  pos_type = pos_type.lower()
  # if pos_list changes, adjust where list is split
  pos_dict = {
      'content': stats[:3],
      'function': stats[3 :],
      'both': stats
  }

  if pos_type in pos_dict:
    for i in pos_dict[pos_type]:
      print(i)
  else:
    print('Please enter "content", "function", or "both" for pos_type.')

#Content word analysis

In [ ]:
incorrect_content = generate_incorrect_lists(content)
incorrect_content[:5]

In [ ]:
count_all_wrong(incorrect_content, content)

In [ ]:
cont_counts = Counter(incorrect_content).most_common()
cont_counts[:5]

In [ ]:
cont_wrong = count_wrong_ln_by_pos(pos_list, cont_counts)
cont_all = count_all_pos(pos_list, y_test)
cont_stats = create_pos_stats(cont_wrong, cont_all)
return_relevant_stats('content', cont_stats)

#Function Word Analysis

In [ ]:
incorrect_function = generate_incorrect_lists(function)
incorrect_function[:5]

In [ ]:
count_all_wrong(incorrect_function, function)

In [ ]:
func_counts = Counter(incorrect_function).most_common()
func_counts[:5]

In [ ]:
func_wrong = count_wrong_ln_by_pos(pos_list, func_counts)
func_all = count_all_pos(pos_list, y_test)
func_stats = create_pos_stats(func_wrong, func_all)
return_relevant_stats('function', func_stats)

# In Training Analysis

In [ ]:
in_train[:5]

In [ ]:
incorrect_in = generate_incorrect_lists(in_train)
count_all_wrong(incorrect_in, in_train)

In [ ]:
in_counts = Counter(incorrect_in).most_common()
in_counts[:5]

In [ ]:
in_wrong = count_wrong_ln_by_pos(pos_list, in_counts)
in_all = count_all_pos(pos_list, in_train)
in_stats = create_pos_stats(in_wrong, in_all)
return_relevant_stats('both', in_stats)

# Out Of Training Analysis

In [ ]:
out_train[:5]

In [ ]:
incorrect_out = generate_incorrect_lists(out_train)
count_all_wrong(incorrect_out, out_train)

In [ ]:
out_counts = Counter(incorrect_out).most_common()
out_counts[:5]

In [ ]:
out_wrong = count_wrong_ln_by_pos(pos_list, out_counts)
out_all = count_all_pos(pos_list, out_train)
out_stats = create_pos_stats(out_wrong, out_all)
return_relevant_stats('both', out_stats)

#Ambiguous Words

In [ ]:
ambiguous_ln[:5]

In [ ]:
incorrect_amb, amb_ln = generate_incorrect_ambiguity_list(ambiguous_ln, ambiguous_words)
count_all_wrong(incorrect_amb, ambiguous_ln)

In [ ]:
amb_counts = Counter(incorrect_amb).most_common()
amb_counts[:5]

In [ ]:
amb_wrong = count_wrong_ln_by_pos(pos_list, amb_counts)
amb_all = count_all_pos(pos_list, ambiguous_ln)
amb_stats = create_pos_stats(amb_wrong, amb_all)
return_relevant_stats('both', amb_stats)

Find the instances of these ln domains and see if the model is simply guessing the most common domain for these ambigous words.

In [ ]:
common_counts = 0
not_common_counts = 0
for i in amb_counts:
  j = incorrect_amb.index(i[0])
  current_dict = amb_ln[j]
  most_common = max(current_dict, key = current_dict.get)
  if most_common == i[0][1]:
    common_counts += i[1]
  else:
    not_common_counts += i[1]

print(f'model incorrectly guessed the most common domain {common_counts} times')
print(f'model incorrectly guessed something besides the most common domain {not_common_counts} times')

#Unambiguous Words

In [ ]:
unambiguous_ln[:5]

In [ ]:
incorrect_unamb = generate_incorrect_lists(unambiguous_ln)
count_all_wrong(incorrect_unamb, unambiguous_ln)

In [ ]:
unamb_counts = Counter(incorrect_unamb).most_common()
unamb_counts[:5]

In [ ]:
unamb_wrong = count_wrong_ln_by_pos(pos_list, unamb_counts)
unamb_all = count_all_pos(pos_list, unambiguous_ln)
unamb_stats = create_pos_stats(unamb_wrong, unamb_all)
return_relevant_stats('both', unamb_stats)

#Unfiltered Test Analsis

In [ ]:
mismatches = []

for i in range(len(y_test)):
  if y_test[i] != predictions[i]:
    a = int_ln_dict[str(y_test[i])]
    b = int_ln_dict[str(predictions[i])]
    mismatches.append(tuple((a, b)))

mismatches[:5]

In [ ]:
all_wrong = generate_incorrect_lists(mismatches)
count_all_wrong(all_wrong, y_test)

In [ ]:
all_counts = Counter(mismatches).most_common()
all_counts[:5]

In [ ]:
wrong_counts = count_wrong_ln_by_pos(pos_list, all_counts)
all_counts = count_all_pos(pos_list, y_test)
all_stats = create_pos_stats(wrong_counts, all_counts)
return_relevant_stats('both', all_stats)